In [3]:
import time
t = time.time()
from torchmetrics.regression import MeanAbsolutePercentageError
import os
import torch
from tsl.metrics.torch import MaskedMSE, MaskedMAE, MaskedMAPE
from tsl.engines import Predictor
import shutil
from Needed_Functions.layers import CITRUS
import networkx as nx
from Needed_Functions.Utilsss import get_evcs_evals
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from tsl.data import SpatioTemporalDataset
import torch.nn as nn
import tsl
import torch
import torch_geometric
import numpy as np
import pandas as pd
from tsl.ops.connectivity import edge_index_to_adj
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler

In [4]:
M = 6
M_hat = M
n_epochs = 300
val_len = 0.1
test_len = 0.1
lr = 1e-2
batch_size = 2048
enable_progress_bar = True
horizon = 12
emb_size = 16    #@param
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2   #@param


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
destination_path = './MetrLA_Results/'

if not os.path.isdir(destination_path):  
    os.mkdir(destination_path)

In [7]:
print(f"torch version: {torch.__version__}")
print(f"  PyG version: {torch_geometric.__version__}")
print(f"  tsl version: {tsl.__version__}")

torch version: 2.1.1+cu118
  PyG version: 2.4.0
  tsl version: 0.9.4


In [8]:
# Plotting functions ###############
pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(edgeitems=3, precision=3)
torch.set_printoptions(edgeitems=2, precision=3)


In [9]:
# Utility functions ################
def print_matrix(matrix):
    return pd.DataFrame(matrix)

In [10]:
#%%
def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

In [11]:
from tsl.datasets import MetrLA

dataset = MetrLA(root='./MetrLA')

print(dataset)

download: 13.1MB [00:00, 15.2MB/s]                            


MetrLA(length=34272, n_nodes=207, n_channels=1)


In [12]:
print(f"Sampling period: {dataset.freq}")
print(f"Has missing values: {dataset.has_mask}")
print(f"Has exogenous variables: {dataset.has_covariates}")
print(f"Covariates: {', '.join(dataset.covariates.keys())}")

print_matrix(dataset.dist)
dataset.dataframe()

Sampling period: <5 * Minutes>
Has missing values: True
Has exogenous variables: True
Covariates: dist


nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-03-01 00:00:00,64.38,67.62,67.12,61.50,66.88,68.75,65.12,67.12,59.62,62.75,...,45.62,65.50,64.50,66.43,66.88,59.38,69.00,59.25,69.00,61.88
2012-03-01 00:05:00,62.67,68.56,65.44,62.44,64.44,68.11,65.00,65.00,57.44,63.33,...,50.67,69.88,66.67,58.56,62.00,61.11,64.44,55.89,68.44,62.88
2012-03-01 00:10:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
2012-03-01 00:15:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
2012-03-01 00:20:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-27 23:35:00,65.00,65.89,68.56,61.67,32.83,54.56,62.44,63.33,59.22,65.33,...,52.89,69.00,65.11,55.67,66.33,62.44,66.78,64.89,69.67,62.33
2012-06-27 23:40:00,61.38,65.62,66.50,62.75,32.83,50.50,62.00,67.00,65.25,67.12,...,54.00,69.25,60.12,60.50,67.25,59.38,66.00,61.25,69.00,62.00
2012-06-27 23:45:00,67.00,59.67,69.56,61.00,32.83,44.78,64.22,63.78,59.78,57.67,...,51.33,67.89,64.33,57.00,66.00,62.67,68.67,63.33,67.44,61.22


In [13]:
print(f"Default similarity: {dataset.similarity_score}")
print(f"Available similarity options: {dataset.similarity_options}")
print("==========================================")

sim = dataset.get_similarity("distance")  # or dataset.compute_similarity()

print("Similarity matrix W:")
print_matrix(sim)

Default similarity: distance
Available similarity options: {'distance'}
Similarity matrix W:


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,1.00,0.39,0.00,0.00,0.00,0.00,0.39,0.00,0.00,...,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00
2,0.00,0.72,1.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,1.00,0.63,0.00,0.01,0.00,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.63,1.00,0.05,0.14,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.08,0.00,0.00,0.00
203,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
204,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.22,...,0.13,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
205,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00


In [14]:
connectivity = dataset.get_connectivity(threshold=0.1,
                                        include_self=False,
                                        layout="edge_index",
                                        force_symmetric=True)

edge_index, edge_weight = connectivity

print(f'edge_index {edge_index.shape}:\n', edge_index)
print(f'edge_weight {edge_weight.shape}:\n', edge_weight)

edge_index (2, 2626):
 [[  0   0   0 ... 206 206 206]
 [ 13  36  37 ... 163 187 198]]
edge_weight (2626,):
 [0.261 0.519 0.509 ... 0.621 0.278 0.649]


In [15]:
from tsl.ops.connectivity import edge_index_to_adj

adj = edge_index_to_adj(edge_index, edge_weight)
print(f'A {adj.shape}:')
print_matrix(adj)
print(f'Sparse edge weights:\n', adj[edge_index[1], edge_index[0]])

A (207, 207):
Sparse edge weights:
 [0.261 0.519 0.509 ... 0.621 0.278 0.649]


In [16]:
torch_dataset = SpatioTemporalDataset(target=dataset.dataframe(),
                                      connectivity=connectivity,
                                      mask=dataset.mask,
                                      horizon=horizon,
                                      window=M_hat,
                                      stride=1)
print(torch_dataset)

SpatioTemporalDataset(n_samples=34255, n_nodes=207, n_channels=1)


In [17]:
sample = torch_dataset[0]
# torch_dataset2 = torch_dataset[:1000]
print(sample)

if sample.has_mask:
    print(sample.mask)
else:
    print("Sample has no mask.")

if sample.has_transform:
    print(sample.transform)
else:
    print("Sample has no transformation functions.")
    
print(sample.pattern)
print("==================   Or we can print patterns and shapes together   ==================")
print(sample)


Data(
  input=(x=[t=6, n=207, f=1], edge_index=[2, e=2626], edge_weight=[e=2626]),
  target=(y=[t=12, n=207, f=1]),
  has_mask=True
)
tensor([[[True],
         [True],
         ...,
         [True],
         [True]],

        [[True],
         [True],
         ...,
         [True],
         [True]],

        ...,

        [[True],
         [True],
         ...,
         [True],
         [True]],

        [[True],
         [True],
         ...,
         [True],
         [True]]])
Sample has no transformation functions.
{'x': 't n f', 'mask': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}
==================   Or we can print patterns and shapes together   ==================
Data(
  input=(x=[t=6, n=207, f=1], edge_index=[2, e=2626], edge_weight=[e=2626]),
  target=(y=[t=12, n=207, f=1]),
  has_mask=True
)


In [18]:
batch = torch_dataset[:5]
print(batch)

StaticBatch(
  input=(x=[b=5, t=6, n=207, f=1], edge_index=[2, e=2626], edge_weight=[e=2626]),
  target=(y=[b=5, t=12, n=207, f=1]),
  has_mask=True
)


In [19]:
# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}


In [20]:
# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=val_len, test_len=test_len)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=batch_size,
)

print(dm)
#%%
dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=2048)
SpatioTemporalDataModule(train_len=27741, val_len=3077, test_len=3425, scalers=[target], batch_size=2048)


In [21]:
input_size = torch_dataset.n_channels   # 1 channel
n_nodes = torch_dataset.n_nodes         # 207 nodes
horizon = torch_dataset.horizon         # 12 time steps

N = [n_nodes, M]
K_list = list(np.array(N)-2)
K_list = [205, M-2]


In [22]:
Graph_List = [nx.from_numpy_array(np.array(adj)), nx.path_graph(N[1])]

evecs, evals, L_list = get_evcs_evals(Graph_List, K_list)

for ii in range(len(evals)):
    evals[ii] = evals[ii].to(device)


[205, 4]


<class 'networkx.utils.decorators.argmap'> compilation 12:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


evecs.shape:,  torch.Size([207, 205])
evecs.shape:,  torch.Size([6, 4])
evecs_kron.shape:,  torch.Size([6, 4])


In [23]:
CGP_GNN = CITRUS(input_size=input_size,
                            n_nodes=n_nodes,
                            horizon=horizon,
                            emb_size=emb_size,
                            hidden_size=hidden_size,
                            rnn_layers=rnn_layers,
                            gnn_kernel=gnn_kernel,
                            edge_index=torch.tensor(edge_index).to(device),
                            edge_weight=torch.tensor(edge_weight).to(device),
                            mass = torch.ones(np.prod(N)).to(device),
                            evals = evals,
                            evecs = torch.tensor(evecs).to(device),
                            C_width = 64,
                            N_block = 3,
                            single_t = True,
                            use_gdc = [],
                            num_nodes = N,
                            last_activation=torch.nn.ReLU(), 
                            mlp_hidden_dims=[64, 64, 64, 64], 
                            dropout=False, 
                            with_MLP=True, 
                            diffusion_method='spectral', 
                            device = device,
                            graph_wise=False)
              
print(CGP_GNN)
print_model_size(CGP_GNN)

CITRUS(
  (node_embeddings): NodeEmbedding(n_nodes=207, embedding_size=16)
  (encoder): Linear(in_features=17, out_features=32, bias=True)
  (CPGNN): CPGNN_ST_in_TTS(
    (last_activation): ReLU()
    (first_lin): Linear(in_features=32, out_features=64, bias=True)
    (last_lin): Linear(in_features=64, out_features=32, bias=True)
    (merge_lin): Linear(in_features=1242, out_features=1, bias=True)
    (node_embeddings): NodeEmbedding(n_nodes=207, embedding_size=4)
    (block_0): CPGNN_block_v2(
      (channel_mixer): Linear(in_features=64, out_features=64, bias=True)
      (diff_derivative): Time_derivative_diffusion_product(
        (Conv_layer): GCN_diff(
          (conv1): GCNConv(64, 64)
        )
      )
      (mlp): MiniMLP(
        (miniMLP_mlp_layer_000): Linear(in_features=128, out_features=64, bias=True)
        (miniMLP_mlp_act_000): ReLU()
        (miniMLP_mlp_layer_001): Linear(in_features=64, out_features=64, bias=True)
        (miniMLP_mlp_act_001): ReLU()
        (miniM

/tmp/ipykernel_1223030/1787871108.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  evecs = torch.tensor(evecs).to(device),
/home/ids/einizade/projects/Imp_on_cluster/CPGNN_Frecasting/GITHUB_codes/Final_Sent_Codes2/MetrLA_PemsBay/Needed_Functions/layers.py:338: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.diffusion_time)


In [24]:
loss_fn = MaskedMAE()

metrics = {'mse': MaskedMSE(),
           'mae': MaskedMAE(),
           'mape': MaskedMAPE()}

In [25]:
# setup predictor_CGP_GNN
# setup predictor
predictor_CGP_GNN = Predictor(
    model=CGP_GNN,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': lr},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics,
# metrics to be logged during train/val/test
)

In [26]:
logger_CGP_GNN = TensorBoardLogger(save_dir="FINAL_MetrLA_Github", name="FINAL_MetrLA_Github", version=0)

checkpoint_callback_CGPGNN = ModelCheckpoint(
    dirpath='FINAL_MetrLA_Github',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

In [27]:
trainer_CGP_GNN = pl.Trainer(max_epochs=n_epochs,
                      logger=logger_CGP_GNN,
                      accelerator=device,
                      devices=1, 
#                      limit_train_batches=train_batches,  # end an epoch after 100 updates
                      callbacks=[checkpoint_callback_CGPGNN],
                      enable_progress_bar=enable_progress_bar)

t_CGPGNN = time.time()
trainer_CGP_GNN.fit(predictor_CGP_GNN, datamodule=dm)
elapsed = time.time() - t_CGPGNN
print('>>>>>>>>>>>>>>>>>>>> CGP-GNN training time, Elapsed: %s' % round(elapsed/60,2), ' minutes')


/home/ids/einizade/anaconda3/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ids/einizade/anaconda3/lib/python3.9/site-pack ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ids/einizade/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/ids/

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ids/einizade/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
Only args ['edge_weight', 'edge_index', 'x'] are forwarded to the model (CITRUS). 
/home/ids/einizade/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/ids/einizade/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the trainin

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.


>>>>>>>>>>>>>>>>>>>> CGP-GNN training time, Elapsed: 39.28  minutes


In [28]:
predictor_CGP_GNN.load_model(checkpoint_callback_CGPGNN.best_model_path)
predictor_CGP_GNN.freeze()

CGP_GNN_results = trainer_CGP_GNN.test(predictor_CGP_GNN, datamodule=dm);


Predictor with already instantiated model is loading a state_dict from /home/ids/einizade/projects/Imp_on_cluster/CPGNN_Frecasting/GITHUB_codes/Final_Sent_Codes2/MetrLA_PemsBay/FINAL_MetrLA_Github/epoch=259-step=3380.ckpt. Cannot  check if model hyperparameters are the same.
/home/ids/einizade/anaconda3/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ids/einizade/anaconda3/lib/python3.9/site-pack ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ids/einizade/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            3.406128168106079
        test_mae            3.3882904052734375
        test_mape           0.09341471642255783
        test_mse             46.51731491088867
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [29]:
elapsed = time.time() - t
print('Elapsed: %s' % round(elapsed/60,2), ' minutes')
print(600*'*')

Elapsed: 39.38  minutes
************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
